<a href="https://colab.research.google.com/github/ake700/Python/blob/main/Author_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import TF 2.x
try:
  # %tensorflow_version only exists in Colab
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
!pip install "tensorflow-text==2.8.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 35.6 MB/s 


In [33]:
import collections
import pathlib
import os

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

print("TF version:", tf.__version__)

# Check for GPU
print("GPU available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

TF version: 2.8.2
GPU available (YESS!!!!)


In [41]:
DIRECTORY_URL = "https://github.com/ake700/Python/tree/main/Data_Science/Text_NeuralNetwork/text_files/"
FILE_NAMES = ['AK.txt', 'AZ.txt', 'SH.txt', 'DK.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

#for name in FILE_NAMES:
  #text_dir = utils.get_file(name, origin=DIRECTORY_PATH + name)

#parent_dir = pathlib.Path(text_dir).parent
#list(parent_dir.iterdir())

'/root/.keras/datasets'

In [42]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)


In [43]:
labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [44]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [45]:
for text, label in all_labeled_data.take(10):
  print("Sentence: ", text.numpy())
  print("Label:", label.numpy())

Sentence:  b'        </tr>'
Label: 0
Sentence:  b'          <td id="LC152" class="blob-code blob-code-inner js-file-line">X ThermoPol buffer . mM of dNTPs . \xc2\xb5M of each primer . U of Taq DNA Polymerase nucleasefree water and  \xc2\xb5L of DNA template. A primer set composed of VKL and VR  was used to amplify the V to V regions of the S rRNA gene . The PCR was conducted in a Biometra TProfessional Basic Thermocycler using the following program: an initial denaturation at  for  min  cycles of / seconds / seconds / seconds and a final extension at  for  min. The size of the PCR amplicon was determined by gel electrophoresis using .  agarose with EXVision Three loading dye and visualized by the ChemiDoc Imaging System. </td>'
Label: 3
Sentence:  b'            name-with-owner="YWtlNzAwL1B5dGhvbg=="'
Label: 3
Sentence:  b'        </tr>'
Label: 0
Sentence:  b'        </tr>'
Label: 0
Sentence:  b'          <td id="L40" class="blob-num js-line-number js-code-nav-line-number js-blob-rnum" 

In [40]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'        <li class="d-block d-md-none">'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'          <td id="LC472" class="blob-code blob-code-inner js-file-line">9 </td>'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b''>, <tf.Tensor: shape=(), dtype=int64, numpy=3>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'        <tr>'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'          <td id="L167" class="blob-num js-line-number js-code-nav-line-number js-blob-rnum" data-line-number="167"></td>'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [46]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [47]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [48]:
tokenized_ds = all_labeled_data.map(tokenize)

In [49]:
for text_batch in tokenized_ds.take(5):
  print("Tokens: ", text_batch.numpy())

Tokens:  [b'</' b'tr' b'>']
Tokens:  [b'<' b'td' b'id' b'="' b'lc' b'152"' b'class' b'="' b'blob' b'-' b'code'
 b'blob' b'-' b'code' b'-' b'inner' b'js' b'-' b'file' b'-' b'line' b'">'
 b'x' b'thermopol' b'buffer' b'.' b'mm' b'of' b'dntps' b'.' b'\xce\xbc'
 b'm' b'of' b'each' b'primer' b'.' b'u' b'of' b'taq' b'dna' b'polymerase'
 b'nucleasefree' b'water' b'and' b'\xce\xbc' b'l' b'of' b'dna' b'template'
 b'.' b'a' b'primer' b'set' b'composed' b'of' b'vkl' b'and' b'vr' b'was'
 b'used' b'to' b'amplify' b'the' b'v' b'to' b'v' b'regions' b'of' b'the'
 b's' b'rrna' b'gene' b'.' b'the' b'pcr' b'was' b'conducted' b'in' b'a'
 b'biometra' b'tprofessional' b'basic' b'thermocycler' b'using' b'the'
 b'following' b'program' b':' b'an' b'initial' b'denaturation' b'at'
 b'for' b'min' b'cycles' b'of' b'/' b'seconds' b'/' b'seconds' b'/'
 b'seconds' b'and' b'a' b'final' b'extension' b'at' b'for' b'min' b'.'
 b'the' b'size' b'of' b'the' b'pcr' b'amplicon' b'was' b'determined' b'by'
 b'gel' b'electrophore

In [51]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [53]:
VOCAB_SIZE = 10000

tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

Vocab size:  9676
First five vocab entries: [b'-', b'="', b'>', b'quot', b'js']


In [54]:
keys = vocab
values = range(2, len(vocab) + 2)  # Reserve `0` for padding, `1` for OOV tokens.

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

In [55]:
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

In [56]:
example_text, example_label = next(iter(all_labeled_data))
print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

Sentence:  b'        </tr>'
Vectorized sentence:  [20 19  4]


In [57]:
all_encoded_data = all_labeled_data.map(preprocess_text)

In [58]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

In [59]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

Text batch shape:  (64, 172)
Label batch shape:  (64,)
First text example:  tf.Tensor(
[20 19  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0], shape=(172,), dtype=int64)
First label example:  tf.Tensor(0, shape=(), dtype=int64)


In [60]:
vocab_size += 2

train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

In [62]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [63]:
model = create_model(vocab_size=vocab_size, num_labels=3)

model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

history = model.fit(train_data, validation_data=validation_data, epochs=3)

Epoch 1/3
147/147 [==============================] - 29s 51ms/step - loss: nan - accuracy: 0.2227 - val_loss: nan - val_accuracy: 0.2274
Epoch 2/3
147/147 [==============================] - 2s 11ms/step - loss: nan - accuracy: 0.2229 - val_loss: nan - val_accuracy: 0.2274
Epoch 3/3
147/147 [==============================] - 2s 11ms/step - loss: nan - accuracy: 0.2229 - val_loss: nan - val_accuracy: 0.2274


In [64]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 2s 5ms/step - loss: nan - accuracy: 0.2274
Loss:  nan
Accuracy: 22.74%


In [66]:
MAX_SEQUENCE_LENGTH = 250

preprocess_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

preprocess_layer.set_vocabulary(vocab)

In [67]:
export_model = tf.keras.Sequential(
    [preprocess_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [68]:
# Create a test dataset of raw strings.
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)

loss, accuracy = export_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 11s 26ms/step - loss: nan - accuracy: 0.2274
Loss:  nan
Accuracy: 22.74%


In [70]:
inputs = [
    "The goal of the SHIME was to reduce C. sakazakii levels through the use of the potential probiotics and synbiotic and to compare the efficacy between the two treatments over a twoweek treatment period",  # Label: 0
    "The survival of L. monocytogenes which was evaluated under simulated human gastric conditions both as single strains and multistrain cocktails was mainly dependent on the gastric acidity and digestion time.",  # Label: 1
    "In comparison the UVC scattering is affected by suspended fat globules and casein micelles in which the protein slightly contributes more than fat.",  # Label: 2
    "the VBNC cells were not able to be resuscitated when following the manufacturer’s recommended storage conditions for reconstituted PIF" # Label 3
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.math.argmax(predicted_scores, axis=1)

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())


Question:  The goal of the SHIME was to reduce C. sakazakii levels through the use of the potential probiotics and synbiotic and to compare the efficacy between the two treatments over a twoweek treatment period
Predicted label:  0
Question:  The survival of L. monocytogenes which was evaluated under simulated human gastric conditions both as single strains and multistrain cocktails was mainly dependent on the gastric acidity and digestion time.
Predicted label:  0
Question:  In comparison the UVC scattering is affected by suspended fat globules and casein micelles in which the protein slightly contributes more than fat.
Predicted label:  0
Question:  the VBNC cells were not able to be resuscitated when following the manufacturer’s recommended storage conditions for reconstituted PIF
Predicted label:  0
